In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import datetime
from ast import literal_eval
import json
import urllib.request

In [2]:
path_alert = r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\alerts_created_copy.csv'
alerts = pd.read_csv(path_alert)

alerts.head(5)

,id,user_id,type,market,currency,data,is_active,is_deleted,max_notifications,notifications,snooze,custom_message,created_at,updated_at,deleted_at,last_checked,last_sent
0,3,1343,price_point,POLONIEX,XRP_USD,"{\comparison\"":\""sell_price\""\""value\"":\""0.32\...",1,0,6.0,4,5min,NaN,1/17/2017 7:45,7/12/2019 22:29,NaN,NaN,NaN
1,5,1344,price_point,BITTREX,BTC_TRIG,"{\comparison\"":\""buy_price\""\""value\"":\"".00000...",0,0,1.0,0,5min,NaN,1/17/2017 8:45,1/17/2017 8:46,NaN,NaN,NaN
2,6,1345,regular_update,BITSTAMP,BTC_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""1day\""}""",0,0,NaN,2,5min,NaN,1/17/2017 8:50,3/31/2017 23:00,NaN,NaN,NaN
3,7,1348,price_point,BITSTAMP,BTC_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""600\""\...",0,1,1.0,2,5min,NaN,1/17/2017 12:11,1/6/2019 3:06,4/29/2018 8:40,NaN,NaN
4,8,1348,price_point,BITSTAMP,BTC_USD,"{\comparison\"":\""buy_price\""\""value\"":\""650\""\...",0,1,1.0,2,5min,NaN,1/17/2017 12:11,4/29/2018 8:40,4/29/2018 8:40,NaN,NaN


In [3]:
alerts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102224 entries, 0 to 102223
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 102224 non-null  int64  
 1   user_id            102224 non-null  int64  
 2   type               102224 non-null  object 
 3   market             102224 non-null  object 
 4   currency           102019 non-null  object 
 5   data               102224 non-null  object 
 6   is_active          102224 non-null  int64  
 7   is_deleted         102224 non-null  int64  
 8   max_notifications  101217 non-null  float64
 9   notifications      102224 non-null  int64  
 10  snooze             102224 non-null  object 
 11  custom_message     0 non-null       float64
 12  created_at         102224 non-null  object 
 13  updated_at         96736 non-null   object 
 14  deleted_at         21076 non-null   object 
 15  last_checked       21320 non-null   object 
 16  la

Let's remove the custom_message and last_sent columns as they are entirely null.

In [4]:
alerts.drop(['last_sent', 'custom_message'], axis=1, inplace=True)
alerts.columns

Index(['id', 'user_id', 'type', 'market', 'currency', 'data', 'is_active',
       'is_deleted', 'max_notifications', 'notifications', 'snooze',
       'created_at', 'updated_at', 'deleted_at', 'last_checked'],
      dtype='object')

In [5]:
alerts.type.unique()

array(['price_point', 'regular_update', 'percent_change'], dtype=object)

There are 3 different types of data as seen above. Each corrisponds to a different amount of information in the data column. If we want to clean the data column then we should split these into 3 different dataframes and then extract only their data column.

In [6]:
alerts_pricepoint = alerts[alerts['type'] == 'price_point']
alerts_regupdate = alerts[alerts['type'] == 'regular_update']
alerts_perchange = alerts[alerts['type'] == 'percent_change']
alerts_regupdate.head()

,id,user_id,type,market,currency,data,is_active,is_deleted,max_notifications,notifications,snooze,created_at,updated_at,deleted_at,last_checked
2,6,1345,regular_update,BITSTAMP,BTC_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""1day\""}""",0,0,NaN,2,5min,1/17/2017 8:50,3/31/2017 23:00,NaN,NaN
40,53,1040,regular_update,BTCE,BTC_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""12h\""}""",0,0,NaN,249,5min,1/18/2017 9:54,5/16/2017 1:55,NaN,NaN
43,56,1327,regular_update,COINBASE,BTC_USD,"{\comparison\"":\""sell_price\""\""value\"":\""1day\""}""",1,0,NaN,1143,5min,1/18/2017 10:23,7/30/2020 23:00,NaN,7/30/2020 23:00
44,57,1375,regular_update,KRAKEN,XBT_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""1day\""}""",0,0,NaN,46,5min,1/18/2017 11:03,5/16/2017 1:55,NaN,NaN
67,93,1385,regular_update,BTCE,BTC_USD,"{\comparison\"":\""buy_price\""\""value\"":\""30min\""}""",0,0,NaN,1690,5min,1/19/2017 0:34,2/25/2017 8:00,NaN,NaN


In [7]:
data_pricepoint = alerts_pricepoint[['data']]
data_regupdate = alerts_regupdate[['data']]
data_perchange = alerts_perchange[['data']]
data_pricepoint.head()

,data
0,"{\comparison\"":\""sell_price\""\""value\"":\""0.32\..."
1,"{\comparison\"":\""buy_price\""\""value\"":\"".00000..."
3,"{\comparison\"":\""buy_price\""\""value\"":\""600\""\..."
4,"{\comparison\"":\""buy_price\""\""value\"":\""650\""\..."
5,"{\comparison\"":\""buy_price\""\""value\"":\""2.5\""\..."


Time to export these files and edit them manually in the CSV

In [8]:
data_pricepoint.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\data_pricepoint')
data_regupdate.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\data_regupdate')
data_perchange.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\data_perchange')

In [9]:
data_pricepoint = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\data_pricepoint_edit.csv', index_col='index')
data_regupdate = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\data_regupdate_edit.csv', index_col='index')
data_perchange = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\data_perchange_edit.csv', index_col='index')

Let's now combine the cleaned Data column with the original dataframes and then delete the old Data column. After that let's make the names a bit cleaner.

In [10]:
data_perchange.head()

,comparison,value,operator,time_period
index,,,,
7,buy_price,"""10""","""<=""","""5min"""
18,buy_price,"""8""",""">=""","""5min"""
19,buy_price,"""8""",""">=""","""1h"""
22,buy_price,"""10""",""">=""","""5min"""
23,buy_price,"""10""",""">=""","""5min"""


In [11]:
alerts_regupdate = alerts_regupdate.join(data_regupdate)
alerts_pricepoint = alerts_pricepoint.join(data_pricepoint)
alerts_perchange = alerts_perchange.join(data_perchange)

In [12]:
alerts_perchange.drop('data', inplace=True, axis=1)
alerts_regupdate.drop('data', inplace=True, axis=1)
alerts_pricepoint.drop('data', inplace=True, axis=1)

In [13]:
alerts_pc = alerts_perchange
alerts_ru = alerts_regupdate
alerts_pp = alerts_pricepoint

In [14]:
alerts_pc.info(), alerts_ru.info(), alerts_pp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20705 entries, 7 to 101993
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 20705 non-null  int64  
 1   user_id            20705 non-null  int64  
 2   type               20705 non-null  object 
 3   market             20705 non-null  object 
 4   currency           20676 non-null  object 
 5   is_active          20705 non-null  int64  
 6   is_deleted         20705 non-null  int64  
 7   max_notifications  20705 non-null  float64
 8   notifications      20705 non-null  int64  
 9   snooze             20705 non-null  object 
 10  created_at         20705 non-null  object 
 11  updated_at         20558 non-null  object 
 12  deleted_at         3109 non-null   object 
 13  last_checked       3798 non-null   object 
 14  comparison         16421 non-null  object 
 15  value              16421 non-null  object 
 16  operator           16

(None, None, None)

Everything looks good here. It seems like most of the alerts are price point alerts. That will be good to know for later. Let's import our notes and clean them all up.

In [15]:
path_note = r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\alerts_notifications_sent.csv'
notes = pd.read_csv(path_note)
notes.head(10)

,id,alert_id,user_id,user_channels,message,message_type,created_at,updated_at
0,1,4,2,1,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 07:55:05,NaN
1,2,4,2,2,Coindera Alert: GDAX - BTC_USD. The buy price...,NaN,2017-01-17 07:55:05,NaN
2,3,4,2,3,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 07:55:05,NaN
3,4,4,2,1,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 08:00:09,NaN
4,5,4,2,3,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 08:00:10,NaN
5,6,1,2,3,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 08:00:11,NaN
6,7,4,2,0,Alert has been paused because it has reached t...,NaN,2017-01-17 08:05:04,NaN
7,8,1,2,3,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 09:00:06,NaN
8,9,1,2,3,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 10:00:07,NaN
9,10,1,2,3,Your GDAX - BTC_USD alert has been triggered. ...,NaN,2017-01-17 11:00:05,NaN


In [16]:
notes.describe()

,id,alert_id,user_id,user_channels,message_type,updated_at
count,3.582507e+06,3.582507e+06,3.582507e+06,3.582507e+06,0.0,0.0
mean,2.091515e+06,6.762906e+04,3.011830e+04,4.464679e+04,NaN,NaN
std,1.449206e+06,4.536166e+04,1.967049e+04,3.076595e+04,NaN,NaN
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
25%,8.956275e+05,3.324100e+04,1.399700e+04,1.842100e+04,NaN,NaN
50%,1.791254e+06,6.085500e+04,2.851400e+04,4.115900e+04,NaN,NaN
75%,3.333776e+06,9.116700e+04,4.602100e+04,6.658500e+04,NaN,NaN
max,4.757038e+06,1.765100e+05,7.618300e+04,1.290240e+05,NaN,NaN


In [17]:
notes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3582507 entries, 0 to 3582506
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             int64  
 1   alert_id       int64  
 2   user_id        int64  
 3   user_channels  int64  
 4   message        object 
 5   message_type   float64
 6   created_at     object 
 7   updated_at     float64
dtypes: float64(2), int64(4), object(2)
memory usage: 218.7+ MB


In [18]:
notes.updated_at.isna().sum(), notes.message_type.isna().sum()

(3582507, 3582507)

It looks like both the updated_at and the message_type columns are entirely full of null values. Let's delete them.

In [19]:
notes.drop(['updated_at', 'message_type'], inplace=True, axis=1)
notes.head()

,id,alert_id,user_id,user_channels,message,created_at
0,1,4,2,1,Your GDAX - BTC_USD alert has been triggered. ...,2017-01-17 07:55:05
1,2,4,2,2,Coindera Alert: GDAX - BTC_USD. The buy price...,2017-01-17 07:55:05
2,3,4,2,3,Your GDAX - BTC_USD alert has been triggered. ...,2017-01-17 07:55:05
3,4,4,2,1,Your GDAX - BTC_USD alert has been triggered. ...,2017-01-17 08:00:09
4,5,4,2,3,Your GDAX - BTC_USD alert has been triggered. ...,2017-01-17 08:00:10


In [20]:
notes.describe()

,id,alert_id,user_id,user_channels
count,3.582507e+06,3.582507e+06,3.582507e+06,3.582507e+06
mean,2.091515e+06,6.762906e+04,3.011830e+04,4.464679e+04
std,1.449206e+06,4.536166e+04,1.967049e+04,3.076595e+04
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,8.956275e+05,3.324100e+04,1.399700e+04,1.842100e+04
50%,1.791254e+06,6.085500e+04,2.851400e+04,4.115900e+04
75%,3.333776e+06,9.116700e+04,4.602100e+04,6.658500e+04
max,4.757038e+06,1.765100e+05,7.618300e+04,1.290240e+05


Everything is looking far better. Let's export our data and move on the exploritory data analysis.

In [21]:
alerts_ru.shape, alerts_pc.shape, alerts_pp.shape

((1007, 16), (20705, 18), (80512, 17))

In [22]:
alerts_final = pd.concat([alerts_ru, alerts_pc, alerts_pp])
alerts_final.type.value_counts()

price_point       80512
percent_change    20705
regular_update     1007
Name: type, dtype: int64

When trying to use the currency column in the EDA we have discovered the need to turn currency into two columns. There is an error in one of the rows. In order to fix this we need to export, edit in the csv, and then re-import the column. Let's do that quickly.

In [23]:
curr_df = alerts_final.currency
curr_df.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\currency.csv')

In [24]:
curr_df_edit = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\currency_edit.csv', index_col=0)
curr_df_edit.head()

,cur_from,cur_into
2,BTC,EUR
40,BTC,EUR
43,BTC,USD
44,XBT,EUR
67,BTC,USD


In [25]:
curr_df_edit.shape, alerts_final.shape

((102224, 2), (102224, 18))

In [26]:
alerts_final = alerts_final.join(curr_df_edit)

In [27]:
alerts_final.head()

,id,user_id,type,market,currency,is_active,is_deleted,max_notifications,notifications,snooze,created_at,updated_at,deleted_at,last_checked,comparison,value,operator,time_period,cur_from,cur_into
2,6,1345,regular_update,BITSTAMP,BTC_EUR,0,0,NaN,2,5min,1/17/2017 8:50,3/31/2017 23:00,NaN,NaN,buy_price,"""1day""",NaN,NaN,BTC,EUR
40,53,1040,regular_update,BTCE,BTC_EUR,0,0,NaN,249,5min,1/18/2017 9:54,5/16/2017 1:55,NaN,NaN,buy_price,"""12h""",NaN,NaN,BTC,EUR
43,56,1327,regular_update,COINBASE,BTC_USD,1,0,NaN,1143,5min,1/18/2017 10:23,7/30/2020 23:00,NaN,7/30/2020 23:00,sell_price,"""1day""",NaN,NaN,BTC,USD
44,57,1375,regular_update,KRAKEN,XBT_EUR,0,0,NaN,46,5min,1/18/2017 11:03,5/16/2017 1:55,NaN,NaN,buy_price,"""1day""",NaN,NaN,XBT,EUR
67,93,1385,regular_update,BTCE,BTC_USD,0,0,NaN,1690,5min,1/19/2017 0:34,2/25/2017 8:00,NaN,NaN,buy_price,"""30min""",NaN,NaN,BTC,USD


Finally let's add our crypto data that we'll be matching all this against.

In [28]:
crypto = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\crypto\bitcoin_per_minute.csv')
crypto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857377 entries, 0 to 4857376
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Timestamp          int64  
 1   Open               float64
 2   High               float64
 3   Low                float64
 4   Close              float64
 5   Volume_(BTC)       float64
 6   Volume_(Currency)  float64
 7   Weighted_Price     float64
dtypes: float64(7), int64(1)
memory usage: 296.5 MB


In [29]:
crypto.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
crypto.describe()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
count,4.857377e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06
mean,1.471301e+09,6.009024e+03,6.013357e+03,6.004488e+03,6.009014e+03,9.323249e+00,4.176284e+04,6.008935e+03
std,8.428019e+07,8.996247e+03,9.003521e+03,8.988778e+03,8.996360e+03,3.054989e+01,1.518248e+05,8.995992e+03
min,1.325318e+09,3.800000e+00,3.800000e+00,1.500000e+00,1.500000e+00,0.000000e+00,0.000000e+00,3.800000e+00
25%,1.398179e+09,4.438600e+02,4.440000e+02,4.435200e+02,4.438600e+02,4.097759e-01,4.521422e+02,4.438306e+02
50%,1.471428e+09,3.596970e+03,3.598190e+03,3.595620e+03,3.597000e+03,1.979811e+00,3.810124e+03,3.596804e+03
75%,1.544288e+09,8.627270e+03,8.632980e+03,8.621090e+03,8.627160e+03,7.278216e+00,2.569821e+04,8.627637e+03
max,1.617149e+09,6.176356e+04,6.178183e+04,6.167355e+04,6.178180e+04,5.853852e+03,1.390067e+07,6.171621e+04


All crypto data looks good! The first few years have a lot of null values but we only need data from 2017 on. Let's convert the time stamp into a datetime object, though for future use.

In [31]:
crypto['date'] = pd.to_datetime(crypto.Timestamp, unit='s', utc=True)
crypto.drop('Timestamp', inplace=True, axis=1)
crypto.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,date
0,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31 07:52:00+00:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:53:00+00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:54:00+00:00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:55:00+00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31 07:56:00+00:00


Everything is looking good now. Let's export everything.

In [32]:
notes.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\EDA_Data\notes.csv')
alerts_final.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\EDA_Data\alerts_final.csv')
crypto.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_3\Data\crypto\bitcoin_per_minute_f.csv')

,id,user_id,type,market,currency,data,is_active,is_deleted,max_notifications,notifications,snooze,custom_message,created_at,updated_at,deleted_at,last_checked,last_sent
0,3,1343,price_point,POLONIEX,XRP_USD,"{\comparison\"":\""sell_price\""\""value\"":\""0.32\...",1,0,6.0,4,5min,NaN,1/17/2017 7:45,7/12/2019 22:29,NaN,NaN,NaN
1,5,1344,price_point,BITTREX,BTC_TRIG,"{\comparison\"":\""buy_price\""\""value\"":\"".00000...",0,0,1.0,0,5min,NaN,1/17/2017 8:45,1/17/2017 8:46,NaN,NaN,NaN
2,6,1345,regular_update,BITSTAMP,BTC_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""1day\""}""",0,0,NaN,2,5min,NaN,1/17/2017 8:50,3/31/2017 23:00,NaN,NaN,NaN
3,7,1348,price_point,BITSTAMP,BTC_EUR,"{\comparison\"":\""buy_price\""\""value\"":\""600\""\...",0,1,1.0,2,5min,NaN,1/17/2017 12:11,1/6/2019 3:06,4/29/2018 8:40,NaN,NaN
4,8,1348,price_point,BITSTAMP,BTC_USD,"{\comparison\"":\""buy_price\""\""value\"":\""650\""\...",0,1,1.0,2,5min,NaN,1/17/2017 12:11,4/29/2018 8:40,4/29/2018 8:40,NaN,NaN
